## `keybert`
* librairie Python pour extraire des mots-clés les plus similaires à un document en exploitant les plongements BERT<br>
⚠️ on doit spécifier la longueur des n-grammes à extraire, alors que l'on ne sait pas quelle est la longueur optimale<br>
⚠️ la grammaticalité des phrases n'est pas prise en compte

([Schopf, 2022](https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db))

In [ ]:
# !pip install 'keybert[all]'

import torch # print(torch.__version__)
import os
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
sentence_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
kw_model = KeyBERT(model=sentence_model)

# Definir les chemins vers les fichiers d'entrée et de sortie
path = './'
file_name = 'autres_keybert_mmr.txt'
file_path = 'corpus_autres.txt'

# Extraction des mots-clés
with open(os.path.join(path, file_path), 'r') as myfile, open(os.path.join(path, file_name), 'w') as outfile:
    raw_data = myfile.readlines()
    start = 0
    end = 300 # diminuer le nb de lignes à traiter si la mémoire RAM est épuisée (p. ex. 10 000)
    while len(raw_data) >= end:
        data = " ".join(raw_data[start:end])
        start = end
        end += 300
        keywords = kw_model.extract_keywords(data, keyphrase_ngram_range=(1, 3), stop_words=None, use_mmr=True, diversity=0.7)
        for k in keywords:
            # print(k) # si mémoire épuisée, commenter cette ligne pour voir
                     # si ça permet au moins d'écrire le contenu dans le fichier
            # Writing each keyword to the output file
            outfile.write(str(k) + '\n')

/home/ljudmila/anaconda3/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-11 13:31:58.315034: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 13:32:00.357430: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 13:32:00.357491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 13:32:00.706706: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS 

In [7]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT

# Initialize KeyBERT
kw_model = KeyBERT()

file_path = '../corpus/test.txt'  # Adjust to the path of your file
output_file_path = '../output/keywords_output.txt'  # The path where you want to save the output

try:
    # Read your own file
    with open(file_path, 'r') as file:
        docs = [file.read()]  # Ensuring it is a list with a single document for this example

    # Extract keywords for each document
    keywords = kw_model.extract_keywords(docs=docs, vectorizer=KeyphraseCountVectorizer())

    # Write the output to a .txt file
    with open(output_file_path, 'w') as output_file:
        for doc_keywords in keywords:
            # Check structure and unpack accordingly
            for keyword_score in doc_keywords:
                keyword, score = keyword_score
                output_file.write(f"{keyword}: {score}\n")
            output_file.write("\n")  # Separate keywords of different documents if necessary

except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: too many values to unpack (expected 2)


In [1]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings

docs = ["""Troubles trophiques consécutifs aux lésions des nerfs.
Sommaire. — Remarques préliminaires. — Objet des conférences de cette année : 
elles seront consacrées à celles des maladies du système nerveux et, en particulier, 
de la moelle épinière, que l'on observe le plus habituelle-ment à la Salpêtrière.
— Troubles de nutrition consécutifs aux lésions de l'axe cérébro-spinal et des nerfs. 
— Ces altérations peuvenl occuper la peau, le tissu cellulaire, les muscles, les articulations, les viscères.
Importance de ces altérations au point de vue du diagnostic et du pronostic. 
— Troubles de nu-trition consécutifs aux lésions des nerfs périphériques. 
— Le système ner-veux, à l'état normal, a peu d'influence sur l'accomplissement des actes nu-tritifs. 
— Les lésions passives des nerfs ou de la moelle ne produisent pas directement de troubles 
trophiques dans les parties périphériques : expé-riences qui le démontrent. — 
Influence de l'irritation et de l'inflammation des nerfs ou des centres nerveux sur la 
production des troubles trophiques. — Les troubles trophiques consécutifs aux lésions 
traumatiques des nerfs, considérés en particulier. — Ils résultent non des sections 
complètes, mais des sections incomplètes, des contusions, etc., des troncs nerveux. 
— Érup-tions cutanées diverses : Érythcme, zona traumalique, pemphigus. — Glossy Skin 
des auteurs anglais. — Lésions musculaires : atrophie. — Lésions ar-ticulaires ; 
lésions osseuses : périostite, nécrose. —Troubles trophiques con-sécutifs aux lésions 
non traumatiques. — Troubles trophiques de l'œil, dans les cas de tumeur comprimant le 
trijumeau. — Inflammation des nerfs spi-naux, consécutive au cancer vertébral, à la 
pachyméningite spinale, à l'as-phyxie par la vapeur de charbon, etc. Eruptions cutanées 
diverses (zona, pemphigus, etc.), atrophie musculaire, arthropathies, qui, en pareil cas, 
se développent en conséquence de la névrite. —Lèpre anesthésique : périné-vrite lépreuse, 
lepra mutilans.
               """]

# Init German KeyBERT model
kw_model = KeyBERT(model=TransformerDocumentEmbeddings('google-bert/bert-base-multilingual-cased'))

# Init vectorizer for the German language
vectorizer = KeyphraseCountVectorizer(spacy_pipeline='fr_core_news_lg', pos_pattern='<N.*><ADJ.*><ADJ.*>+', stop_words='french')

# Get German keyphrases
kw_model.extract_keywords(docs=docs, vectorizer=vectorizer)

/home/ljudmila/anaconda3/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-02 10:31:39.946865: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 10:31:39.997156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 10:31:39.997189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 10:31:39.998741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS 

[('tions cutanées diverses', 0.8186),
 ('troubles trophiques consécutifs', 0.6976)]